In [1]:
import json
import oandapyV20
from oandapyV20 import API    # the client
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.pricing as pricing
import pandas as pd
import numpy as np
import mplcyberpunk
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
plt.style.use("cyberpunk")

In [2]:
ACCESS_TOKEN = "4aa51e7711418ba1aa356b835fab3afd-53be4f4d5aa21768bf421b354527dc96"
ACCOUNT_ID = "101-004-16909090-001"
client = API(access_token=ACCESS_TOKEN)

In [9]:
def get_prices(count: int, from_date: str, instrument: str = "EUR_USD", granularity: str = "H1"):  
  params = {
    "count": count,
    "granularity": granularity,
    "from": from_date,
    }
  r = instruments.InstrumentsCandles(instrument=instrument, params=params)
  return client.request(r)


def make_df(data: dict):
  data = pd.DataFrame(data["candles"])
  candles = pd.json_normalize(data["mid"]).rename(columns={"o": "open", "h": "high", "l": "low", "c": "close"})
  data = data.join(candles, how = "left").drop(["mid"], axis = 1)
  data["time"] = pd.to_datetime(data["time"])
  data = data.apply(lambda x: x.astype(float) if x.name in ["open", "high", "low", "close"] else x, axis = 0)
  return data

def get_from_time(hour_diff: int = 5, day_diff: int = 0):
  if datetime.now().weekday() not in [5, 6] or day_diff != 0:
    return (datetime.now().replace(microsecond=0,second=0,minute=0) - timedelta(hours=hour_diff) - timedelta(days=day_diff)).strftime("%Y-%m-%dT%H:%M:%S.%fZ")
  else: return "Market is closed on weekends."


In [31]:
from_time = get_from_time(day_diff=2)
print(from_time)
data = make_df(
    get_prices(5, from_time)
    )

2024-05-24T09:00:00.000000Z


In [32]:
data

,complete,volume,time,open,high,low,close
0,True,1641,2024-05-24 09:00:00+00:00,1.08246,1.08318,1.08237,1.08288
1,True,1835,2024-05-24 10:00:00+00:00,1.08285,1.08443,1.08276,1.08404
2,True,1932,2024-05-24 11:00:00+00:00,1.08402,1.08430,1.08345,1.08409
3,True,4697,2024-05-24 12:00:00+00:00,1.08410,1.08538,1.08376,1.08513
4,True,4424,2024-05-24 13:00:00+00:00,1.08514,1.08536,1.08420,1.08426


In [29]:
data["spread"] = data["close"].diff()
data["spread"].mean()

0.00023111111111109552

In [103]:
datetime.now()

datetime.datetime(2024, 5, 26, 12, 6, 12, 305405)

In [17]:
get_from_time(day_diff=1)

'2024-05-25T07:00:00.000000Z'